<a href="https://colab.research.google.com/github/hydanggia2899/Shopee_ProductDetection/blob/master/Shopee_Week2_VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Shopee Week 2 - Product Detection

## 1.Install Libraries and download the dataset

In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from torch.utils.data.dataset import Dataset
import numpy as np
import pandas as pd
import torchvision.models as models
import torch.nn as nn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from PIL import Image,ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm.notebook import tqdm

In [ ]:
#Download the dataset by GDrive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '13Kwb-19DS9CLSUOMokhsjKtutQXSDUT5'})
download.GetContentFile('shopee_new.zip')

In [ ]:
!7za x shopee_new.zip -o\dataset


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 10069790881 bytes (9604 MiB)

Extracting archive: shopee_new.zip
  3% 4096 Open              --
Path = shopee_new.zip
Type = zip
Physical Size = 10069790881
64-bit = +

  0%      0% 92 - test/test/0200edbbc50ac2e6479dd950af864b19.jpg                                                          0% 184 - test/test/045c74e62b891cb781ff56f4ad70ce9e.jpg                                                           0% 267 - test/test/05c77efdfc219514c1fcdf95a11639bd.jp

In [ ]:
Qdownload = drive.CreateFile({'id': '1dwsnSj7mHmDEqZ1SDNuxh9EO1CYmdSXn'})
download.GetContentFile('train.zip')

### Remove Extra Images if possible

In [ ]:
!7za x train.zip -o\dataset

In [ ]:
submit = pd.read_csv("submission.csv")

In [ ]:
submit['category'] = submit.category.apply(lambda c: str(c).zfill(2))

In [ ]:
submit.to_csv("submit_5.csv",index = False)

In [ ]:
import shutil  
shutil.rmtree("dataset/train_ok")

In [ ]:
submit

,filename,category
0,0004b03ad7eabfb3989727c461310a84.jpg,12
1,0004d34271d0a0253f8a95d0746bae05.jpg,16
2,000d4ad3d0a372fd6eea2fee6bf738ef.jpg,21
3,0012f125ad07123e69468ae1d1d59de4.jpg,22
4,0015831b249b856e569e883a9dc6d4fa.jpg,12
...,...,...
12181,ffef3b0e7505855ba5794ebab9aff719.jpg,00
12182,ffef86f7872cd020992e00b7cf6585c7.jpg,17
12183,fff8fdfc178e3e94414fc10c69271f88.jpg,10
12184,fff9ee1d7f920b920dd603631d24bc07.jpg,23


In [ ]:
path = "dataset/"
TRAIN_PATH = path + "train/train/"
TEST_PATH =path + "test/test/"

In [ ]:
def remove_extra(path_csv,path_files):
  #Check folder contains enough dataset
  import glob
  import os.path as osp
  test_sum = pd.read_csv(path_csv)
  list_test = glob.glob(path_files+ "*.jpg")
  for index,item in enumerate(list_test):
    list_test[index] = osp.basename(item)
  #print(list_test)
  count = 0
  extra_file = list()
  for file in (list_test):
    if(file not in test_sum["filename"].tolist()):
      count+=1
      extra_file.append(file)
  print(extra_file)
  #Remove extra file
  import os
  count = 0
  for filename in extra_file:
    os.remove(path_files + filename)
    print("File Removed!")
    count +=1
  print(count)

In [ ]:
#Remove extra
remove_extra(path + "test.csv",path + "test/test/")

['10425d5faff26f90d07e3582700829e0 (1).jpg', '019e994abbd6a49c1846995a480b7883 (1).jpg', '2b738be035c8674db9fb2fc0c09b69ce (1).jpg', 'abd26f08c2fba98d905d0145661e457e (1).jpg', '105b0d039887573a8266511ba42a7723 (1).jpg', '4347e61279df8af5978c15ec5425676b (1).jpg']
File Removed!
File Removed!
File Removed!
File Removed!
File Removed!
File Removed!
6


## 2. Work with the dataset

In [ ]:
class ShopeeDataset(Dataset):
    def __init__(self,directory,transform,train=True):
        self.transform = transform                      # function that turns an image into a tensor that can be passed to the pytorch model
        self.img_ls = list()                            # where the path to each training image file is stored

        classes = ["%.2d" % i for i in range(0,len(os.listdir(directory)))]   
        print(classes)        # ['00','01',...,'41'] i.e. the folders for the images of each corresponding category
        for clas in classes:
            path = os.path.join(directory,clas)
            ls = sorted(os.listdir(path))               # obtain list of image file names in a class folder
            if train:
                for img_name in ls[:-50]:                                           # take all images except the last 50 images for each category. This will be used as training data
                    self.img_ls.append((os.path.join(path,img_name),int(clas)))     # adds to img_ls in a tuple format (path,category)
            else:
                for img_name in ls[-50:]:                                           # last 50 images of each category for validation set, for monitoring of current performance
                    self.img_ls.append((os.path.join(path,img_name),int(clas)))
        
    def __getitem__(self, idx):                         # this function will be called by pytorch dataloader class.
        name, label = self.img_ls[idx]
        img = Image.open(name).convert('RGB')           # load image
        img.load()
        img = self.transform(img)                       # apply transformation to turn image into torch tensor
        return {"image": img, "label": label}
    
    def __len__(self):                                  # allows usage of len(<name of dataset>) to obtain size of entire dataset
        return len(self.img_ls)

In [ ]:
def train_epoch(model,  trainloader,  criterion, device, optimizer):


    model.train()
    losses = list()
    ct=0
    for batch_idx, data in enumerate(tqdm(trainloader)):       #iterate through dataset. tqdm is the loading bar, allows you to track progress
        ct+=1
        inputs=data['image'].to(device)                 # images
        labels=data['label'].to(device)                 # labels
        optimizer.zero_grad()
        outputs = model(inputs)                         # prediction
        loss = criterion(outputs, labels)               # loss function, tells you how "wrong" your predictions are based on the training dataset.
        loss.backward()                                 # backpropogation
        optimizer.step()                                # gradient descent
        losses.append(loss.item())

    return sum(losses)/ct

In [ ]:
def evaluate(model, dataloader, device):

    model.eval()                                                    # set model to evaluation mode
    total=0                                                         # store total number of validation samples
    correct = 0                                                     # store total number of correct prediction samples
    with torch.no_grad():
      for ctr, data in enumerate(dataloader):
          inputs = data['image'].to(device)                         # obtain validation images
          outputs = model(inputs)                                   # pass images to model
          labels = data['label']                                    # actual labels for validation images. Also known as ground truth
          labels = labels.float()
          cpuout= outputs.to('cpu')
          total += len(labels)                                      # total number of samples

          pred = cpuout.argmax(dim=1, keepdim=True)                 # for each sample, the model gives a score for each category. Choose the category that contains the highest score as the prediction
          correct += pred.eq(labels.view_as(pred)).sum().item()     # number of correct predictions

      accuracy = correct / total                                    # accuracy = total correct / total number of samples

    return accuracy

In [ ]:
def train_modelcv(dataloader_cvtrain, dataloader_cvtest ,  model ,  criterion, optimizer, scheduler, num_epochs, device):

  for epoch in range(num_epochs):                                                           # iterates through specified number of epochs
    print('-' * 10)
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)

    model.train(True)                                                                           
    train_loss=train_epoch(model,  dataloader_cvtrain,  criterion,  device , optimizer )    # train model

    model.train(False)
    measure = evaluate(model, dataloader_cvtest, device)                                    # evaluate model

    print('Top 1 Accuracy:', measure, "\n")                                                 # print accuracy for current epoch
  return None

## 3. Train model. 
#### VGG19

In [ ]:
!sudo find / -name ".DS_Store" -depth -exec rm {} \;

find: warning: you have specified the -depth option after a non-option argument -name, but options are not positional (-depth affects tests specified before it as well as those specified after it).  Please specify options before other arguments.



In [ ]:
from collections import OrderedDict
batchsize=64                                                            # number of images being passed to the model at any one time. If you face cuda memory error, reduce batchsize.
lr = 0.0001                                                               # how "fast" you want your model to learn. Very important to play around with different values.

data_transform = transforms.Compose([transforms.Resize((224,224)),      # images are of different size, reshape them to same size. Up to you to decide what size to use.
                                     transforms.RandomCrop(size=(224,224),padding=(10,10)),
                                     transforms.RandomHorizontalFlip(p=0.5),
                                     transforms.RandomRotation(degrees=15,fill=0),
                                     transforms.ToTensor(),             # convert image to torch tensor
                                     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])   # normalizing images helps improve convergence

                                                # where your training images are
dataset_train = ShopeeDataset(TRAIN_PATH, data_transform,train=True)          # training dataset
dataset_valid = ShopeeDataset(TRAIN_PATH, data_transform,train=False)         # validation dataset to measure current model performance. (rough gauge)

loadertr=torch.utils.data.DataLoader(dataset_train,batch_size=batchsize,shuffle=True)       # dataloader for training set
loaderval=torch.utils.data.DataLoader(dataset_valid,batch_size=batchsize,shuffle=True)      # dataloader for validation set

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   # which device to use. If GPU is detected, use GPU, else, use CPU
criterion = nn.CrossEntropyLoss()                                       # Loss function
#callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

model = models.resnet18(pretrained=True)                                # transfer learning, Pytorch has a pretrained model trained on ImageNet. Turn this off if you want to, it may or may not help in the end. Helps achieve higher accuracy with less epochs though
ct = 0

for name, child in model.named_children():
    ct += 1
    if ct < 7:
        for name2, params in child.named_parameters():
          params.requires_grad = False
#fc = nn.Sequential(OrderedDict([('fc1',nn.Linear(512,256)),('relu', nn.ReLU()),('fc2',nn.Linear(256,42)),('output',nn.LogSoftmax(dim=1))]))                               # transfer learning, Pytorch has a pretrained model trained on ImageNet. Turn this off if you want to, it may or may not help in the end. Helps achieve higher accuracy with less epochs though
model.fc = nn.Linear(512,42)                                         # as we only have 42 classes, change the last layer to predict on 42 classes.
model.to(device)                                         # as we only have 42 classes, change the last layer to predict on 42 classes.
                                                      # send model to device, and hopefully your device is GPU

optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
maxnumepochs=15                                                        # number of epochs to train on. Should most definitely increase this to train more.
train_modelcv(dataloader_cvtrain = loadertr,                            # lets start training
              dataloader_cvtest = loaderval,  
              model = model,  
              criterion = criterion, 
              optimizer = optimizer,
              scheduler = None,
              num_epochs = maxnumepochs,
              device = device)

['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41']
['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41']
----------
Epoch 0/14
----------



Top 1 Accuracy: 0.700952380952381 

----------
Epoch 1/14
----------



Top 1 Accuracy: 0.7209523809523809 

----------
Epoch 2/14
----------



Top 1 Accuracy: 0.7447619047619047 

----------
Epoch 3/14
----------



Top 1 Accuracy: 0.7342857142857143 

----------
Epoch 4/14
----------


...


Top 1 Accuracy: 0.7442857142857143 

----------
Epoch 11/14
----------



Top 1 Accuracy: 0.7452380952380953 

----------
Epoch 12/14
----------



Top 1 Accuracy: 0.7466666666666667 

----------
Epoch 13/14
----------



Top 1 Accuracy: 0.7447619047619047 

----------
Epoch 14/14
----------



Top 1 Accuracy: 0.7380952380952381 



In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from torch.utils.data.dataset import Dataset
class ShopeeTestDataset(Dataset):
    def __init__(self,directory,transform):
        self.transform = transform
        self.img_ls = list()

        ls = sorted(os.listdir(directory))
        for img_name in ls:
            self.img_ls.append((os.path.join(directory,img_name)))
        
    def __getitem__(self, idx):
        name = self.img_ls[idx]
        img = Image.open(name).convert('RGB')
        img.load()
        img = self.transform(img)
        return img,name[12:]
    
    def __len__(self):
        return len(self.img_ls)

In [ ]:
def inference(model, dataloader, device):
    # df = pd.DataFrame()
    path_ls = list()
    pred_ls = list()
    model.eval()
    with torch.no_grad():
      for ctr, data in enumerate(dataloader):
          inputs,path = data
          inputs = inputs.to(device)
          outputs = model(inputs)
          cpuout= outputs.to('cpu')
          pred = cpuout.argmax(dim=1).numpy()
          path_ls += list(path)
          pred_ls += list(pred)
    df = pd.DataFrame({'filename':path_ls,'category':pred_ls})

    return df

In [ ]:
import torch
test_dataset = ShopeeTestDataset("dataset/test/test", data_transform)
loaderte=torch.utils.data.DataLoader(test_dataset,batch_size=256,shuffle=False)
df = inference(vgg,loaderte,device)                               # predictions are stored in this dataframe
df.to_csv("test_prediction.csv",index=False)      # export dataframe as csv. It will export to your Google drive home folder

In [ ]:
import pandas as pd 
import numpy as np
#test = pd.read_csv("test.csv")
predict = pd.read_csv("test_prediction.csv")

In [ ]:
for i in range(0,len(predict["filename"])):
  predict["filename"][i] = predict["filename"][i].replace("/test/","")

In [ ]:
predict.to_csv("submit_2.csv",index = False)